In [ ]:
import pandas as pd
import os

In [ ]:
original = pd.read_csv("서울특별시의회 회의록 제303회[정례회](2021.11.01 ~ 2021.12.27).csv", encoding = 'euc-kr')
original

In [ ]:
loc = os.getcwd()
join_dir = os.path.join(loc, "질문답변자 조인")
train = pd.read_excel(os.path.join(join_dir, "train.xlsx"))
train

In [ ]:
test = pd.read_csv(os.path.join(join_dir, "2021_감정분석_test_ver2.csv"), encoding = 'euc-kr')
test

In [ ]:
train_test = pd.concat([train, test])
train_test

In [ ]:
# 발언 내용이 unique key의 역할을 할 수 없지만, 다른 key를 추출할 수 없어 병합 후 중복 제거하는 방식으로 진행.
# 회의일자, 발언 순번을 key로 이용하는 방식으로 보완 가능. 
result = pd.merge(original, train_test, how = 'left', on = ['발언내용'])
result

In [ ]:
result.to_csv("2021감성분석_중복제거.csv", encoding = 'euc-kr')

In [ ]:
result['flag'] = True
result

In [ ]:
for i in range(1, len(result)):
    if (result.iloc[i - 1, 6] == result.iloc[i, 6]) & (result.iloc[i - 1, 8] == result.iloc[i, 8]):
        result.iloc[i, 12] = False
result_final = result[result['flag']]
result_final

In [ ]:
result_final.to_csv("2021_감성분석_최종.csv", index = False, encoding = 'euc-kr')

In [ ]:
# 학습 데이터 셋 선정을 위한 결과 비교

In [ ]:
result_part = pd.read_csv(os.path.join(join_dir, "2020_감정분석_test.csv"), encoding = "euc-kr")
result_all = pd.read_csv(os.path.join(join_dir, "2020_감정분석_test_all_ver.csv"), encoding = "euc-kr")

In [ ]:
diff = []
for i in range(len(result_part)):
    if result_part.iloc[i, 1] != result_all.iloc[i, 1]:
        diff.append([result_part.iloc[i, 0], result_part.iloc[i, 1], result_all.iloc[i, 1]])
diff_df = pd.DataFrame(diff, columns = ['발언내용', '2021일부', '2021전체'])
diff_df.to_csv(os.path.join(join_dir, "2020 학습결과 비교.csv"), index = False, encoding = 'euc-kr')

In [ ]:
result_final = pd.read_csv("2021_감성분석_통합본.csv", encoding = 'euc-kr')
result_final

In [ ]:
person = list(result_final['발언자'].unique())
person

In [ ]:
d = []
for p in person:
    tmp = result_final[result_final['발언자'] == p]
    pos = len(tmp[tmp['감정'] == '긍정'])
    neu = len(tmp[tmp['감정'] == '중립'])
    neg = len(tmp[tmp['감정'] == '부정'])
    try:
        ratio = (pos / (pos + neg)) * 100
    except ZeroDivisionError:
        ratio = 0
    d.append([p, pos, neu, neg, ratio])
print(d)

In [ ]:
sent_ratio = pd.DataFrame(d, columns = ['발언자', '긍정', '중립', '부정', '감성지수'])
sent_ratio['감성지수'] = sent_ratio['감성지수'].apply(round)

In [ ]:
sent_ratio.to_csv("2021_감성지수_산출_중립포함.csv", index = False, encoding = 'euc-kr')
# 중복 위원 있음.